## Convolutional Neural Networks

Again, I am not going to even try to do a better job than this post...:

https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/

Now, let's review some of the famous CNN architectures on starting on p.367 of Hands on Machine Learning.

## Python Conv Net

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, BatchNormalization, BatchNormalization, Dropout
from keras.datasets import mnist
from sklearn.metrics import confusion_matrix
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

def scale_image(images):
    images = images.reshape(images.shape[0], 28, 28, 1)
    images = images.astype('float32')
    scaled_images = images / 255
    return scaled_images

x_train = scale_image(x_train)
x_test = scale_image(x_test)

11280384/11490434 [============================>.] - ETA: 0s

In [39]:
x_train.shape

(60000, 28, 28, 1)

In [19]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), activation='elu', input_shape=(28, 28, 1), kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, kernel_size=(5, 5), activation='elu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation='elu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='elu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
batch_normalization_5 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 64)          0         
__________

In [21]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy')

In [22]:
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 15s - loss: 0.6339 - val_loss: 0.1071
Epoch 2/5
48000/48000 [==============================] - 14s - loss: 0.2244 - val_loss: 0.0817
Epoch 3/5
48000/48000 [==============================] - 13s - loss: 0.1679 - val_loss: 0.0570
Epoch 4/5
48000/48000 [==============================] - 13s - loss: 0.1423 - val_loss: 0.0518
Epoch 5/5
48000/48000 [==============================] - 13s - loss: 0.1320 - val_loss: 0.0499


In [23]:
test_predictions = np.argmax(model.predict(x_test),1)

In [24]:
confusion_matrix(y_test, test_predictions)

array([[ 976,    0,    0,    0,    0,    0,    2,    1,    1,    0],
       [   0, 1130,    0,    1,    0,    1,    1,    2,    0,    0],
       [   1,    1, 1022,    0,    0,    0,    1,    3,    4,    0],
       [   0,    0,    2,  997,    0,    1,    0,    5,    3,    2],
       [   0,    0,    1,    0,  974,    0,    2,    0,    0,    5],
       [   2,    0,    0,   14,    0,  866,    3,    2,    3,    2],
       [   7,    2,    1,    0,    1,    1,  942,    0,    4,    0],
       [   0,    4,    5,    1,    0,    0,    0, 1016,    1,    1],
       [   6,    0,    2,    1,    1,    0,    1,    3,  959,    1],
       [   1,    2,    0,    5,    8,    0,    0,    6,    2,  985]])

In [25]:
np.sum(y_test == test_predictions) / test_predictions.shape

array([ 0.9867])

## Pre-training and Data Augmentation

Source: https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975

In [20]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(48,48,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(10, activation='sigmoid'))

model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


Model loaded.


In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________